# Advent of code

### day 1

In [20]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
1000
2000
3000

4000

5000
6000

7000
8000
9000

10000
"""
)
f = open("day01.txt")
elfs = (
    pd.Series(f)
    .rename("calories")
    .str.strip()
    .loc[lambda s: s != ""]
    .astype(int)
    .to_frame()
    .reset_index()
    .assign(elf=lambda df: df["index"] - df.index)
    [["elf", "calories"]]
    .groupby(by="elf")
    .sum()
)

print(f"The elf with the most calories has {elfs.max().to_numpy()}")

top3 = (
    elfs
    .sort_values("calories", ascending=False)
    .iloc[:3]
)

print(f"The top 3 elves have {top3.sum().to_numpy()}")


The elf with the most calories has [69693]
The top 3 elves have [200945]


# day 2

In [33]:
from io import StringIO

import pandas as pd
import numpy as np

"""
The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors

The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors

1 for Rock, 2 for Paper, and 3 for Scissors

0 if you lost, 3 if the round was a draw, and 6 if you won
"""


f = StringIO(
"""
A Y
B X
C Z
"""
)
f = open("day02.txt")

tournament = (
    pd.read_csv(f, sep=" ", header=None)
    .set_axis(["foe", "me"], axis=1)
    .replace({
        "A": 1,
        "B": 2,
        "C": 3,
        "X": 1,
        "Y": 2,
        "Z": 3,
    })
    .assign(outcome=lambda df: ((df["me"] - df["foe"] + 1) % 3) * 3)
)

print(f"The total score using the guide: {tournament.sum().loc[['me', 'outcome']].sum()}")

"""
X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win
"""

tournament2 = (
    tournament
    .assign(outcome=lambda df: 3 * (df["me"] - 1))
    .assign(me=lambda df: (df["me"] + df["foe"]) % 3 + 1)
)

print(f"The total score using the 2nd guide: {tournament2.sum().loc[['me', 'outcome']].sum()}")


The total score using the guide: 12740
The total score using the 2nd guide: 11980


# day 3

In [69]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
""".strip()
)
f = open("day03.txt")
total = 0
for l in f:
    ll = l.strip()
    aa = set(ll[:len(ll) // 2])
    bb = set(ll[len(ll) // 2:])
    oo = ord((aa & bb).pop())
    total += oo - 96 if oo > 95 else oo - 65 + 27

print(f"The sum to priorities is {total}")

f = StringIO(
"""
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
""".strip()
)
f = open("day03.txt")
total = 0
lines = f.readlines()
for a, b, c in zip(lines[0::3], lines[1::3], lines[2::3]):
    badge = (set(a.strip()) & set(b.strip()) & set(c.strip())).pop()
    oo = ord(badge)
    total += oo - 96 if oo > 95 else oo - 65 + 27
    
print(f"The sum to the badges priorities is {total}")
    

The sum to priorities is 7980
The sum to the badges priorities is 2881


# day 4

In [90]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
""".strip()
)
f = open("day04.txt")

cleaning = (
    pd.read_csv(
        StringIO(f.read().replace("-", ",")),
        header=None)
    .set_axis(["a", "b", "c", "d"], axis=1)
    .assign(
        is_one_range=lambda df:
        ((df["a"] <= df["c"]) & (df["d"] <= df["b"])) |
        ((df["a"] >= df["c"]) & (df["d"] >= df["b"])))
)

print(f"""
The number of pairs where one range contains the other is {cleaning["is_one_range"].astype(int).sum()}
""".strip())

cleaning2 = (
    cleaning
    .assign(
        is_overlap=lambda df:
        ((df["a"] <= df["c"]) & (df["c"] <= df["b"])) |
        ((df["c"] <= df["a"]) & (df["a"] <= df["d"])))
)

print(f"""
The number of pairs where one range overlaps with the other is {cleaning2["is_overlap"].astype(int).sum()}
""".strip())


The number of pairs where one range contains the other is 450
The number of pairs where one range overlaps with the other is 837


# day 5

In [173]:
from copy import deepcopy
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
""".strip("\n")
)
f = open("  day05.txt  ".strip())
lines = f.readlines()
moves = list(
    pd.DataFrame([l.strip().split() for l in lines if "move" in l])
    .iloc[:, 1::2]
    .astype(int)
    .itertuples(name=None, index=None)
)
crates0 = (
    pd.DataFrame([list(l) for l in lines if not "move" in l][:-2])
    .iloc[:, 1::4]
    .agg("".join, axis=0)
    .apply(reversed)
    .apply("".join)
    .apply(str.strip)
    .apply(list)
    .tolist()
)

crates = [None] + deepcopy(crates0)
for n, s, e in moves:
    for _ in range(n):
        crates[e].append(crates[s].pop())


print(f"""
The crates at the top of the stacks are {"".join([c[-1] for c in crates[1:]])}
""".strip())

crates = [None] + deepcopy(crates0)
for n, s, e in moves:
    crates[e] += crates[s][-n:]
    crates[s] = crates[s][:-n]


print(f"""
Using the crate mover 9001 the crates at the top of the stacks are {"".join([c[-1] for c in crates[1:]])}
""".strip())

The crates at the top of the stacks are RNZLFZSJH
Using the crate mover 9001 the crates at the top of the stacks are CNSFCGJSM


# day 6

In [180]:

from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
bvwbjplbgvbhsrlpgdmjqwftvncz
""".strip("\n")
)
f = open("  day06.txt  ".strip())
word = f.read().strip()
ans = None
for i in range(len(word)):
    if len(set(word[max(0, i - 4):i])) == 4:
        ans = i
        break

print(f"""
The marker is after character {ans}
""".strip())


ans = None
for i in range(len(word)):
    if len(set(word[max(0, i - 14):i])) == 14:
        ans = i
        break

print(f"""
The start-of-message marker is after character {ans}
""".strip())

The marker is after character 1892
The start-of-message marker is after character 2313


# template